# One to one

In [1]:
import sys
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
sys.path.append("../")
from src.service.provider import ProviderService
from operator import itemgetter
from langsmith.run_helpers import traceable

provider = ProviderService()
provider.config.enable_tracing(project="LEARN")

TEMPLATE = """
Câu hỏi và tiêu đề có liên quan nhau không? nếu có thì trả lời true, không thì false.
Câu hỏi: '{question}'
Tiêu đề: '{title}'
"""


def build_filter_chain(provider: ProviderService):
    """
        Build a filter chain to filter relevant retrieved documents to user query.
    """
    template = 'Is the "{question}" relevant to the "{title}"? true or fasle.'
    prompt = ChatPromptTemplate.from_template(template=template)
    chain = ({
        "question": itemgetter("question"),
        "title": itemgetter("title")
    }
    | prompt
    | provider.get_simple_gemini_pro())
    return chain

chain = build_filter_chain(provider=provider)

bot_gemini = provider.get_simple_gemini_pro()


@traceable(tags=["optimize", "question_relevancy"])
def run_bot(question, title):
    prompt = f'Is the "{question}" relevant to the "{title}"? true or fasle.'
    return bot_gemini(prompt)

@traceable(tags=["optimize", "question_relevancy"])
def run_chain(question, title):
    return chain.invoke({"question": question, "title": title})

/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


From (44, 5) to 44
From (11, 4) to 11


In [2]:
q = "điểm chuẩn ngành phần mềm năm 2023 là bao nhiêu"
t = "Ngành Kỹ thuật phần mềm"
a = run_chain(q, t)
a

'True'

# Many to one

In [15]:
@traceable(tags=["optimize", "question_relevancy"])
def run_bot(question, title1, title2):
    prompt=f"""Do not output more than one word
    Title 1: {title1}
    Title 2: {title2}
    Which title is more relevant to the "{question}". Output "one" or "two"."""
    return bot_gemini(prompt)


In [17]:
question = "Cho em hỏi về ngành Khoa học máy tính"
t1 = "Ngành Kỹ thuật phần mềm"
t2 = "Khoa học máy tính"

a = run_bot(question, t1, t2)
a

'two'